# Bogumił Kamiński, 2019-03-25

In [1]:
using DataFrames

In [2]:
lines = readlines("sample_dataframes.log")

22855-element Array{String,1}:
 "commit ef530285cc6991660feac33d71f332e1a7e34686"                                
 "Author: Bogumił Kamiński <bkamins@sgh.waw.pl>"                                  
 "Date:   Wed Mar 13 21:17:48 2019 +0100"                                         
 ""                                                                               
 "    upgrade documentation (#1748)"                                              
 ""                                                                               
 " src/groupeddataframe/grouping.jl | 153 ++++++++++++++++++++++++++++++++-------"
 " 1 file changed, 125 insertions(+), 28 deletions(-)"                            
 ""                                                                               
 "commit b69c5980109a49ea3bfe6234173a8db16f8e2cba"                                
 "Author: Milan Bouchet-Valat <nalimilan@club.fr>"                                
 "Date:   Wed Mar 13 16:01:16 2019 +0100"               

In [3]:
author = r"^Author: ([^<]*) <"
insc = r"^.+changed, ([0-9]+) insertion"
delc = r"^.+changed.*, ([0-9]+) deletion"

r"^.+changed.*, ([0-9]+) deletion"

In [4]:
authordata = DataFrame(author=String[], action=String[], count=Int[])

,author,action,count
,String,String,Int64


In [5]:
curauthor = ""

""

In [6]:
for line in lines
    m = match(author, line)
    if m !== nothing
        global curauthor = m[1]
    end
    m = match(insc, line)
    m === nothing || push!(authordata,
                           [curauthor, "insertion", parse(Int, m[1])])
    m = match(delc, line)
    m === nothing || push!(authordata,
                           [curauthor, "deletion", parse(Int, m[1])])
end

In [7]:
authorstats = by(authordata, [:author, :action], count = :count=>sum)

,author,action,count
,String,String,Int64
1,Bogumił Kamiński,insertion,12717
2,Bogumił Kamiński,deletion,9104
3,Milan Bouchet-Valat,insertion,9720
4,Milan Bouchet-Valat,deletion,8765
5,pdeffebach,insertion,499
6,pdeffebach,deletion,631
7,Harry Scholes,insertion,41
8,Harry Scholes,deletion,27
9,Katharine Hyatt,insertion,27


In [8]:
df = unstack(authorstats, :action, :count)

,author,deletion,insertion
,String,Int64⍰,Int64⍰
1,Adam Labadorf,1,1
2,Alain Lichnewsky,1,1
3,AlainLich,1,1
4,Alex Arslan,1112,197
5,Alex Mellnik,100,164
6,Alexey Stukalov,466,1284
7,Andreas Noack,627,455
8,Andreas Noack Jensen,67,212
9,Andy Ferris,3,4


In [9]:
df.all = coalesce.(df.deletion, 0) .+ coalesce.(df.insertion, 0)

150-element Array{Int64,1}:
    2
    2
    2
 1309
  264
 1750
 1082
  279
    7
   27
    2
    4
    4
    ⋮
  140
  520
 1130
   28
 3361
  139
    2
    2
  850
   18
    2
  593

In [10]:
sort!(df, :all, rev=true)

,author,deletion,insertion,all
,String,Int64⍰,Int64⍰,Int64
1,John Myles White,31288,105657,136945
2,Cameron Prybol,73847,3486,77333
3,Tom Short,10039,12975,23014
4,Bogumił Kamiński,9104,12717,21821
5,Milan Bouchet-Valat,8765,9720,18485
6,Sean Garborg,9991,4008,13999
7,Harlan Harris,1898,4514,6412
8,Douglas Bates,1700,2087,3787
9,quinnj,1604,1757,3361
